# Multi LInear Regression

## Data: FinalDataset.csv
## X: Urban spatial data of 500m around a subway station
## Y: From Fast Fourier Transformation, magnitude of each frequecy of every 10 interver from 0 to 100


In [1]:
import pandas as pd
import torch
from torch import nn
from torch import optim
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import numpy as np
from sklearn.base import BaseEstimator
from torch.utils.data import DataLoader, TensorDataset
import pyro
import pyro.distributions as dist
from pyro.nn import PyroModule, PyroSample
from pyro.infer import MCMC, NUTS
import pyro.distributions.constraints as constraints
from pyro.infer.autoguide import AutoMultivariateNormal, init_to_mean
from pyro.infer import SVI, Trace_ELBO, Predictive
import pyro.optim as optim

In [3]:
date_types = ['wd_sc', 'wd_hc', 'we_sc', 'we_hc']
targets = ['min_ride', 'max_ride']
clusters = [['석촌', '태릉입구', '홍대입구', '강남', '선릉', '역삼', '교대', '공덕', '수유', '상봉', '중화', '상도']]

def prepare_counts_df(predictive):
    counts = predictive['coefs']
    counts_mean = counts.mean()
    counts_std = counts.std()

    mean_dict = {'mean': counts_mean}
    std_dict = {'std': counts_std}

    mean_df = pd.DataFrame(mean_dict, index = [0])
    std_df = pd.DataFrame(std_dict, index = [0])

    return mean_df, std_df

for date_type in date_types:
    df = pd.read_csv(f'../Data/240103_data/train_{date_type}.csv', index_col = 0)
    for target in targets:
        x = df.iloc[:, 2:-8]
        x = x.fillna(0)
        if target == 'min_ride':
            y = df[target]
        elif target == 'max_ride':
            y = np.log(df[target])
            target = 'log_max_ride'
        for cluster_number in range(len(clusters)):
            test_stations = clusters[cluster_number]
            test_id = df[df['name'].isin(test_stations)].index
            train_id = df[~df['name'].isin(test_stations)].index
            x_test = x.loc[test_id, :]
            x_train = x.loc[train_id, :]
            y_test = y.loc[test_id]
            y_train = y.loc[train_id]
            y_test_np = y_test
            y_train_np = y_train

            x_train = torch.tensor(x_train.values, dtype = torch.float)
            x_test = torch.tensor(x_test.values, dtype = torch.float)
            y_train = torch.tensor(y_train_np.values, dtype = torch.float)
            y_test = torch.tensor(y_test_np.values, dtype = torch.float)

            torch.cuda.empty_cache()

            array_y_test = y_test.numpy()
            y_test = pd.DataFrame(array_y_test)
            y_test.to_csv(f'../Result/240116_LastCluster/Cluster{cluster_number+1}/test_{target}_{date_type}.csv')

            class BNN(nn.Module):
                def __init__(self, input_dim = 17, output_dim = 1, prior_scale = 10.):
                    super(BNN, self).__init__()
                    
                    self.activation = nn.ReLU()
                    self.layer1 = PyroModule[nn.Linear](input_dim, output_dim)
                    self.input_dim = input_dim
                    self.output_dim = output_dim

                    # Set Layer parameters as random variables
                    self.layer1.weight1 = PyroSample(dist.Normal(0., prior_scale).expand([output_dim, input_dim]).to_event(2))
                    self.layer1.bias1 = PyroSample(dist.Normal(0., prior_scale).expand([output_dim]).to_event(1))
                    
                def forward(self, x, y = None):
                    mu = self.activation(self.layer1(x)).squeeze(-1)
                    sigma = pyro.sample("sigma", dist.Gamma(.5, 1))
                    with pyro.plate("data", size = mu.shape[0]):
                        pyro.sample("coefs", dist.Normal(mu, sigma * sigma), obs = y)
                    #return mu

            model = BNN()
            pyro.set_rng_seed(42)

            
            ###svi
            guide = AutoMultivariateNormal(model, init_loc_fn = init_to_mean)
            svi = SVI(model, guide, optim.Adam({"lr": .01}), loss = Trace_ELBO(vectorize_particles = True))

            # Convert data to PyTorch tensors
            x_train = np.array(x_train)
            y_train = np.array(y_train)
            x_train = torch.from_numpy(x_train).float()
            y_train = torch.from_numpy(y_train).float()

            pyro.clear_param_store()
            num_iters = 30000
            for i in range(num_iters):
                elbo = svi.step(x_train, y_train)
                if i % 500 == 0:
                    print("elbo loss: {}".format(elbo))

            x_test = np.array(x_test)

            x_test = torch.from_numpy(x_test).float()
            y_test = np.array(y_test)
            y_test = torch.from_numpy(y_test).float()

            mean_dfs = []
            std_dfs = []
            for i in range(len(x_test)):
                pred_svi = Predictive(model, guide = guide, num_samples = 5000)(x_test[i].reshape(1, 17))
                mean_df_, std_df_ = prepare_counts_df(pred_svi)
                mean_dfs.append(mean_df_)
                std_dfs.append(std_df_)

            mean_concat = pd.concat(mean_dfs, axis = 0)
            std_concat = pd.concat(std_dfs, axis = 0)

            mean_concat.to_csv(f'../Result/240116_LastCluster/Cluster{cluster_number+1}/pred_{target}_{date_type}_mean.csv')
            std_concat.to_csv(f'../Result/240116_LastCluster/Cluster{cluster_number+1}/pred_{target}_{date_type}_std.csv')


elbo loss: 10426438.794331133
elbo loss: 229159.79158104956
elbo loss: 27337.9087305665
elbo loss: 19283.86172556877
elbo loss: 14545.925424039364
elbo loss: 13861.835511505604
elbo loss: 8830.813659667969
elbo loss: 7283.879822969437
elbo loss: 3459.7282358407974
elbo loss: 2857.507586836815
elbo loss: 2085.7125931978226
elbo loss: 1846.1097767353058
elbo loss: 1917.0328586101532
elbo loss: 1654.5804405212402
elbo loss: 1496.9228782653809
elbo loss: 1617.3575872182846
elbo loss: 1310.5873589515686
elbo loss: 1270.7773673534393
elbo loss: 1269.835916876793
elbo loss: 1235.9197667837143
elbo loss: 1218.3066148757935
elbo loss: 1218.6463344097137
elbo loss: 1228.0793342590332
elbo loss: 1217.9160171747208
elbo loss: 1215.3825497627258
elbo loss: 1219.4228105545044
elbo loss: 1212.5256447792053
elbo loss: 1210.5537691116333
elbo loss: 1207.1943620443344
elbo loss: 1204.5195100307465
elbo loss: 1201.15027654171
elbo loss: 1199.4208145141602
elbo loss: 1188.5569610595703
elbo loss: 1181.902

In [14]:
guide(x_test[0].reshape(1, 17), y_test[0].reshape(1, 12))

RuntimeError: shape '[1, 12]' is invalid for input of size 1

In [45]:
pred_svi = Predictive(model, guide = guide, num_samples = 5000)(x_test[0].reshape(1, 17))
for k, v in pred_svi.items():
    print(f"{k}: {tuple(v.shape)}")

sigma: (5000, 1)
coefs: (5000, 1)


In [121]:
def prepare_counts_df(predictive):
    counts = predictive['coefs']
    counts_mean = counts.mean()
    counts_std = counts.std()

    mean_dict = {'mean': counts_mean}
    std_dict = {'std': counts_std}

    mean_df = pd.DataFrame(mean_dict, index = [0])
    std_df = pd.DataFrame(std_dict, index = [0])

    return mean_df, std_df

mean_dfs = []
std_dfs = []
for i in range(len(x_test)):
    pred_svi = Predictive(model, guide = guide, num_samples = 5000)(x_test[i].reshape(1, 17))
    mean_df_, std_df_ = prepare_counts_df(pred_svi)
    mean_dfs.append(mean_df_)
    std_dfs.append(std_df_)

mean_concat = pd.concat(mean_dfs, axis = 0)
std_concat = pd.concat(std_dfs, axis = 0)

In [115]:
mean_concat.to_csv(f'../Result/240103_Multi+Bayesian Quantity/pred_{target}_{date_type}_mean.csv')
std_concat.to_csv(f'../Result/240103_Multi+Bayesian Quantity/pred_{target}_{date_type}_std.csv')

In [97]:
class BNN(nn.Module):
    def __init__(self, input_dim = 15, output_dim = 1, prior_scale = 10.):
        super(BNN, self).__init__()
        
        self.activation = nn.ReLU()
        self.layer1 = PyroModule[nn.Linear](input_dim, output_dim)
        self.input_dim = input_dim
        self.output_dim = output_dim

        # Set Layer parameters as random variables
        self.layer1.weight1 = PyroSample(dist.Normal(0., prior_scale).expand([output_dim, input_dim]).to_event(2))
        self.layer1.bias1 = PyroSample(dist.Normal(0., prior_scale).expand([output_dim]).to_event(1))
        
    def forward(self, x, y = None):
        mu = self.activation(self.layer1(x)).squeeze(-1)
        sigma = pyro.sample("sigma", dist.Gamma(.5, 1))
        with pyro.plate("data", size = mu.shape[0]):
            coef = pyro.sample("coefs", dist.Normal(mu, sigma * sigma), obs = y)
        return coef

model = BNN()
pyro.set_rng_seed(42)

In [98]:
nuts_kernel = NUTS(model, jit_compile=False)
mcmc = MCMC(nuts_kernel, num_samples=1000)
mcmc.run(x_train, y_train)

hmc_samples = {k: v.detach().cpu().numpy() for k, v in mcmc.get_samples().items()}

Sample: 100%|██████████| 2000/2000 [00:05, 337.32it/s, step size=1.00e+00, acc. prob=0.935]


In [99]:
mcmc.summary()


                mean       std    median      5.0%     95.0%     n_eff     r_hat
     sigma      2.80      0.07      2.80      2.69      2.90    409.45      1.00

Number of divergences: 0


In [75]:
hmc_samples.keys()

dict_keys(['sigma'])

In [ ]:
mean_concat = pd.concat(mean_dfs, axis = 0)
std_concat = pd.concat(std_dfs, axis = 0)

In [117]:
pyro.poutine.trace(guide).get_trace(x_test[0].reshape(1, 15), y_test[0].reshape(1, 12))

In [ ]:
class BNN(nn.Module):
    def __init__(self, input_dim = 15, hidden_dim = 4, output_dim = 12, prior_scale = 10.):
        super(BNN, self).__init__()
        
        self.activation = nn.ReLU()
        self.layer1 = PyroModule[nn.Linear](input_dim, hidden_dim)
        self.layer2 = PyroModule[nn.Linear](hidden_dim, output_dim)
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.hidden_dim = hidden_dim

        # Set Layer parameters as random variables
        self.layer1.weight1 = PyroSample(dist.Normal(0., prior_scale).expand([hidden_dim, input_dim]).to_event(2))
        self.layer1.bias1 = PyroSample(dist.Normal(0., prior_scale).expand([hidden_dim]).to_event(1))
        self.layer2.weight2 = PyroSample(dist.Normal(0., prior_scale).expand([output_dim, hidden_dim]).to_event(2))
        self.layer2.bias2 = PyroSample(dist.Normal(0., prior_scale).expand([output_dim]).to_event(1))

    def forward(self, x, y = None):
        x = self.activation(self.layer1(x))
        mu = self.layer2(x).squeeze(-1)
        sigma = pyro.sample("sigma", dist.Gamma(.5, 1))
        with pyro.plate("data", mu.shape[0]):
            obs = pyro.sample("obs", dist.Normal(mu, sigma * sigma).to_event(1), obs = y)
        return mu
    '''
        with pyro.irange("data", mu.shape[0]):
            return pyro.sample("obs", dist.MultivariateNormal(loc, scale_tril), obs = y)'''
        #return mu

model = BNN()
pyro.set_rng_seed(42)

In [118]:
nuts_kernel = NUTS(model, jit_compile=False)
mcmc = MCMC(nuts_kernel, num_samples=1000)
mcmc.run(x_train, y_train)

Warmup:   0%|          | 0/2000 [00:00, ?it/s]

NotImplementedError: HMC/NUTS does not support model with subsample sites.

In [ ]:
predictive = Predictive(model=model, posterior_samples=mcmc.get_samples())
preds = predictive(x_test)
preds

In [ ]:
x_test.shape

In [ ]:
pred['obs'].shape

In [ ]:

predictive = Predictive(model, guide = guide, num_samples = 1000)

In [ ]:
x_test = np.array(x_test)
x_test = torch.from_numpy(x_test).float()
y_test = np.array(y_test)
y_test = torch.from_numpy(y_test).float()

pred = predictive(x_test, y_test)

In [ ]:
from pyro.infer import Predictive


num_samples = 1000
predictive = Predictive(model, guide=guide, num_samples=num_samples)
svi_samples = {k: v.reshape(num_samples).detach().cpu().numpy()
               for k, v in predictive(x_train, y_train).items()
               if k != "obs"
               }

In [ ]:
svi_samples.keys()

In [ ]:
for site, values in svi_samples.items():
    print("Site: {}".format(site))
    print(values, "\n")

In [ ]:
predictive(x_train, y_train).keys()

In [ ]:
import pyro
import pyro.distributions as dist
import torch
import torch.nn as nn

class MultivariateNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(MultivariateNN, self).__init__()
        self.linear1 = nn.Linear(input_dim, hidden_dim)
        self.linear2 = nn.Linear(hidden_dim, output_dim)
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.hidden_dim = hidden_dim

    def forward(self, x):
        with pyro.plate("data", x.shape[0]):
            # Sample hidden layer weights
            hidden_weights = pyro.sample("hidden_weights", dist.Normal(torch.zeros(self.input_dim, self.hidden_dim), torch.ones(self.input_dim, self.hidden_dim)))
            # Apply activation function to first layer
            hidden = torch.relu(self.linear1(x) @ hidden_weights)

            # Sample output layer weights
            output_weights = pyro.sample("output_weights", dist.Normal(torch.zeros(self.hidden_dim, self.output_dim), torch.ones(self.hidden_dim, self.output_dim)))
            # Calculate outputs
            outputs = self.linear2(hidden) @ output_weights

            # Define likelihood function
            pyro.sample("obs", dist.MultivariateNormal(outputs, torch.eye(self.output_dim)))

# Define data and model
x = torch.randn(10, 3)
y = torch.randn(10, 2)
model = MultivariateNN(3, 5, 2)

# Train the model
optimizer = torch.optim.Adam(model.parameters())
for epoch in range(10):
    optimizer.zero_grad()
    loss = pyro.infer.TraceMeanField_ELBO().loss(model, x, y)
    loss.backward()
    optimizer.step()

# Predict on new data
new_x = torch.randn(1, 3)
with torch.no_grad():
    predictions = model(new_x)
    print(f"Predictions for new data: {predictions}")


In [ ]:
model = BNN()
pyro.set_rng_seed(42)

nuts_kernel = NUTS(model, jit_compile = True)

mcmc = MCMC(nuts_kernel, num_samples = 100)

# Convert data to PyTorch tensors
x_train = np.array(x_train)
y_train = np.array(y_train)
x_train = torch.from_numpy(x_train).float()
y_train = torch.from_numpy(y_train).float()

mcmc.run(x_train)
#posterior_samples = mcmc.get_samples()

In [ ]:
class BNN(nn.Module):
    def __init__(self, input_dim = 15, hidden_dim = 4, output_dim = 12, prior_scale = 10.):
        super().__init__()
        
        self.activation = nn.ReLU()
        self.layer1 = PyroModule[nn.Linear](input_dim, hidden_dim)
        self.layer2 = PyroModule[nn.Linear](hidden_dim, output_dim)

        # Set Layer parameters as random variables
        self.layer1.weight1 = PyroSample(dist.Normal(0., prior_scale).expand([hidden_dim, input_dim]).to_event(2))
        self.layer1.bias1 = PyroSample(dist.Normal(0., prior_scale).expand([hidden_dim]).to_event(1))
        self.layer2.weight2 = PyroSample(dist.Normal(0., prior_scale).expand([output_dim, hidden_dim]).to_event(2))
        self.layer2.bias2 = PyroSample(dist.Normal(0., prior_scale).expand([output_dim]).to_event(1))
    
    def forward(self, x, y = None):
        x = self.activation(self.layer1(x))
        mu = self.layer2(x).squeeze(-1)
        sigma = pyro.sample("sigma", dist.Gamma(.5, 1))
        with pyro.plate("data", mu.shape[0]):
            pyro.sample("obs", dist.Normal(mu.shape[-1], sigma), obs = y)
        return mu

model = BNN()
guide = AutoDiagonalNormal(model)



In [ ]:
pyro.clear_param_store()
for j in range(num_iterations):
    # calculate the loss and take a gradient step
    loss = svi.step(x_train, y_train)
    if j % 100 == 0:
        print("[iteration %04d] loss: %.4f" % (j + 1, loss / len(data)))

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt

# Set random seed for reproducibility
np.random.seed(42)

# Generate data
x_obs = np.hstack([np.linspace(-0.2, 0.2, 500), np.linspace(0.6, 1, 500)])
noise = 0.02 * np.random.randn(x_obs.shape[0])
y_obs = x_obs + 0.3 * np.sin(2 * np.pi * (x_obs + noise)) + 0.3 * np.sin(4 * np.pi * (x_obs + noise)) + noise

x_true = np.linspace(-0.5, 1.5, 1000)
y_true = x_true + 0.3 * np.sin(2 * np.pi * x_true) + 0.3 * np.sin(4 * np.pi * x_true)

# Set plot limits and labels
xlims = [-0.5, 1.5]
ylims = [-1.5, 2.5]

# Create plot
fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(x_true, y_true, 'b-', linewidth=3, label="True function")
ax.plot(x_obs, y_obs, 'ko', markersize=4, label="Observations")
ax.set_xlim(xlims)
ax.set_ylim(ylims)
ax.set_xlabel("X", fontsize=30)
ax.set_ylabel("Y", fontsize=30)
ax.legend(loc=4, fontsize=15, frameon=False)

plt.show()

In [ ]:
import pyro
import pyro.distributions as dist
from pyro.nn import PyroModule, PyroSample
import torch.nn as nn


class MyFirstBNN(PyroModule):
    def __init__(self, in_dim=1, out_dim=1, hid_dim=5, prior_scale=10.):
        super().__init__()

        self.activation = nn.Tanh()  # or nn.ReLU()
        self.layer1 = PyroModule[nn.Linear](in_dim, hid_dim)  # Input to hidden layer
        self.layer2 = PyroModule[nn.Linear](hid_dim, out_dim)  # Hidden to output layer

        # Set layer parameters as random variables
        self.layer1.weight = PyroSample(dist.Normal(0., prior_scale).expand([hid_dim, in_dim]).to_event(2))
        self.layer1.bias = PyroSample(dist.Normal(0., prior_scale).expand([hid_dim]).to_event(1))
        self.layer2.weight = PyroSample(dist.Normal(0., prior_scale).expand([out_dim, hid_dim]).to_event(2))
        self.layer2.bias = PyroSample(dist.Normal(0., prior_scale).expand([out_dim]).to_event(1))

    def forward(self, x, y=None):
        x = x.reshape(-1, 1)
        x = self.activation(self.layer1(x))
        mu = self.layer2(x).squeeze()
        sigma = pyro.sample("sigma", dist.Gamma(.5, 1))  # Infer the response noise

        # Sampling model
        with pyro.plate("data", x.shape[0]):
            obs = pyro.sample("obs", dist.Normal(mu, sigma * sigma), obs=y)
        return mu
    
from pyro.infer import MCMC, NUTS

model = MyFirstBNN()

# Set Pyro random seed
pyro.set_rng_seed(42)

# Define Hamiltonian Monte Carlo (HMC) kernel
# NUTS = "No-U-Turn Sampler" (https://arxiv.org/abs/1111.4246), gives HMC an adaptive step size
nuts_kernel = NUTS(model, jit_compile=False)  # jit_compile=True is faster but requires PyTorch 1.6+

# Define MCMC sampler, get 50 posterior samples
mcmc = MCMC(nuts_kernel, num_samples=50)

# Convert data to PyTorch tensors
x_train = torch.from_numpy(x_obs).float()
y_train = torch.from_numpy(y_obs).float()

# Run MCMC
mcmc.run(x_train, y_train)

In [ ]:
from pyro.infer import Predictive

predictive = Predictive(model=model, posterior_samples=mcmc.get_samples())
x_test = torch.linspace(xlims[0], xlims[1], 3000)
preds = predictive(x_test)

def plot_predictions(preds):
    y_pred = preds['obs'].T.detach().numpy().mean(axis=1)
    y_std = preds['obs'].T.detach().numpy().std(axis=1)

    fig, ax = plt.subplots(figsize=(10, 5))
    xlims = [-0.5, 1.5]
    ylims = [-1.5, 2.5]
    plt.xlim(xlims)
    plt.ylim(ylims)
    plt.xlabel("X", fontsize=30)
    plt.ylabel("Y", fontsize=30)

    ax.plot(x_true, y_true, 'b-', linewidth=3, label="true function")
    ax.plot(x_obs, y_obs, 'ko', markersize=4, label="observations")
    ax.plot(x_obs, y_obs, 'ko', markersize=3)
    ax.plot(x_test, y_pred, '-', linewidth=3, color="#408765", label="predictive mean")
    ax.fill_between(x_test, y_pred - 2 * y_std, y_pred + 2 * y_std, alpha=0.6, color='#86cfac', zorder=5)

    plt.legend(loc=4, fontsize=15, frameon=False)

plot_predictions(preds)

In [ ]:
import os
import torch
import torch.nn.functional as F
import pyro
import pyro.distributions as dist
import pyro.distributions.constraints as constraints

smoke_test = ('CI' in os.environ)
assert pyro.__version__.startswith('1.8.6')

def model(data):
    m = pyro.sample("m", dist.Normal(0, 1))
    sd = pyro.sample("sd", dist.LogNormal(m, 1))
    with pyro.plate("N", len(data)):
        pyro.sample("obs", dist.Normal(m, sd), obs=data)

data = torch.ones(10)
pyro.render_model(model, model_args=(data,))

graph = pyro.render_model(model, model_args=(data,), filename="model.pdf")

In [ ]:
import os
from functools import partial
import torch
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import pyro
import pyro.distributions as dist

# for CI testing
smoke_test = ('CI' in os.environ)
assert pyro.__version__.startswith('1.8.6')
pyro.set_rng_seed(1)


# Set matplotlib settings
%matplotlib inline
plt.style.use('default')

In [ ]:
DATA_URL = "https://d2hg8soec8ck9v.cloudfront.net/datasets/rugged_data.csv"
data = pd.read_csv(DATA_URL, encoding="ISO-8859-1")
df = data[["cont_africa", "rugged", "rgdppc_2000"]]
df = df[np.isfinite(df.rgdppc_2000)]
df["rgdppc_2000"] = np.log(df["rgdppc_2000"])

In [ ]:
from torch import nn
from pyro.nn import PyroModule

assert issubclass(PyroModule[nn.Linear], nn.Linear)
assert issubclass(PyroModule[nn.Linear], PyroModule)

In [ ]:
# Dataset: Add a feature to capture the interaction between "cont_africa" and "rugged"
df["cont_africa_x_rugged"] = df["cont_africa"] * df["rugged"]
data = torch.tensor(df[["cont_africa", "rugged", "cont_africa_x_rugged", "rgdppc_2000"]].values,
                        dtype=torch.float)
x_data, y_data = data[:, :-1], data[:, -1]

linear_reg_model = PyroModule[nn.Linear](3, 1)

loss_fn = torch.nn.MSELoss(reduction = 'sum')
optim = torch.optim.Adam(linear_reg_model.parameters(), lr = 0.05)
num_iterations = 1500 if not smoke_test else 2

def train():
    # run the model forward on the data
    y_pred = linear_reg_model(x_data).squeeze(-1)
    # calculate the mse loss
    loss = loss_fn(y_pred, y_data)
    # initialize gradients to zero
    optim.zero_grad()
    # backpropagate
    loss.backward()
    # take a gradient step
    optim.step()
    return loss

for j in range(num_iterations):
    loss = train()
    if (j + 1) % 50 == 0:
        print("[iteration %04d] loss: %.4f" % (j + 1, loss.item()))


# Inspect learned parameters
print("Learned parameters:")
for name, param in linear_reg_model.named_parameters():
    print(name, param.data.numpy())

In [ ]:
from pyro.nn import PyroSample
from pyro.infer.autoguide import AutoDiagonalNormal
from pyro.infer import SVI, Trace_ELBO

class BayesianRegression(PyroModule):
    def __init__(self, in_features, out_features):
        super().__init__()
        self.linear = PyroModule[nn.Linear](in_features, out_features)
        self.linear.weight = PyroSample(dist.Normal(0., 1.).expand([out_features, in_features]).to_event(2))
        self.linear.bias = PyroSample(dist.Normal(0., 10.).expand([out_features]).to_event(1))

    def forward(self, x, y = None):
        sigma = pyro.sample("sigma", dist.Uniform(0., 10.))
        mean = self.linear(x).squeeze(-1)
        with pyro.plate("data", x.shape[0]):
            obs = pyro.sample("obs", dist.Normal(mean, sigma), obs = y)
        return mean

model = BayesianRegression(3, 1)
guide = AutoDiagonalNormal(model)

adam = pyro.optim.Adam({"lr": 0.03})
svi = SVI(model, guide, adam, loss = Trace_ELBO())

pyro.clear_param_store()
for j in range(num_iterations):
    # calculate the loss and take a gradient step
    loss = svi.step(x_data, y_data)
    if j % 100 == 0:
        print("[iteration %04d] loss: %.4f" % (j + 1, loss / len(data)))

In [ ]:
guide.requires_grad_(False)

for name, value in pyro.get_param_store().items():
    print(name, pyro.param(name))

In [ ]:
guide.quantiles([0.25, 0.5, 0.75])

In [ ]:
from pyro.infer import Predictive


def summary(samples):
    site_stats = {}
    for k, v in samples.items():
        site_stats[k] = {
            "mean": torch.mean(v, 0),
            "std": torch.std(v, 0),
            "5%": v.kthvalue(int(len(v) * 0.05), dim=0)[0],
            "95%": v.kthvalue(int(len(v) * 0.95), dim=0)[0],
        }
    return site_stats


predictive = Predictive(model, guide=guide, num_samples=800,
                        return_sites=("linear.weight", "obs", "_RETURN"))
samples = predictive(x_data)
pred_summary = summary(samples)

mu = pred_summary["_RETURN"]
y = pred_summary["obs"]
predictions = pd.DataFrame({
    "cont_africa": x_data[:, 0],
    "rugged": x_data[:, 1],
    "mu_mean": mu["mean"],
    "mu_perc_5": mu["5%"],
    "mu_perc_95": mu["95%"],
    "y_mean": y["mean"],
    "y_perc_5": y["5%"],
    "y_perc_95": y["95%"],
    "true_gdp": y_data,
})

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(12, 6), sharey=True)
african_nations = predictions[predictions["cont_africa"] == 1]
non_african_nations = predictions[predictions["cont_africa"] == 0]
african_nations = african_nations.sort_values(by=["rugged"])
non_african_nations = non_african_nations.sort_values(by=["rugged"])
fig.suptitle("Regression line 90% CI", fontsize=16)
ax[0].plot(non_african_nations["rugged"],
           non_african_nations["mu_mean"])
ax[0].fill_between(non_african_nations["rugged"],
                   non_african_nations["mu_perc_5"],
                   non_african_nations["mu_perc_95"],
                   alpha=0.5)
ax[0].plot(non_african_nations["rugged"],
           non_african_nations["true_gdp"],
           "o")
ax[0].set(xlabel="Terrain Ruggedness Index",
          ylabel="log GDP (2000)",
          title="Non African Nations")
idx = np.argsort(african_nations["rugged"])
ax[1].plot(african_nations["rugged"],
           african_nations["mu_mean"])
ax[1].fill_between(african_nations["rugged"],
                   african_nations["mu_perc_5"],
                   african_nations["mu_perc_95"],
                   alpha=0.5)
ax[1].plot(african_nations["rugged"],
           african_nations["true_gdp"],
           "o")
ax[1].set(xlabel="Terrain Ruggedness Index",
          ylabel="log GDP (2000)",
          title="African Nations");

In [ ]:
weight = samples["linear.weight"]
weight = weight.reshape(weight.shape[0], 3)
gamma_within_africa = weight[:, 1] + weight[:, 2]
gamma_outside_africa = weight[:, 1]
fig = plt.figure(figsize=(10, 6))
sns.distplot(gamma_within_africa, kde_kws={"label": "African nations"},)
sns.distplot(gamma_outside_africa, kde_kws={"label": "Non-African nations"})
fig.suptitle("Density of Slope : log(GDP) vs. Terrain Ruggedness");

In [ ]:
import logging
import os

import torch
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from torch.distributions import constraints

import pyro
import pyro.distributions as dist
import pyro.optim as optim

In [ ]:
def model(is_cont_africa, ruggedness, log_gdp):
    a = pyro.sample("a", dist.Normal(0., 10.))
    b_a = pyro.sample("bA", dist.Normal(0., 1.))
    b_r = pyro.sample("bR", dist.Normal(0., 1.))
    b_ar = pyro.sample("bAR", dist.Normal(0., 1.))
    sigma = pyro.sample("sigma", dist.Uniform(0., 10.))
    mean = a + b_a*is_cont_africa + b_r*ruggedness + b_ar*is_cont_africa*ruggedness
    with pyro.plate("data", len(ruggedness)):
        pyro.sample("obs", dist.Normal(mean, sigma), obs = log_gdp)

def guide(is_cont_africa, ruggedness, log_gdp):
    a_loc = pyro.param('a_loc', torch.tensor(0.))
    a_scale = pyro.param('a_scale', torch.tensor(1.), constraint = constraints.positive)
    sigma_loc = pyro.param('sigma_loc', torch.tensor(1.), constraint = constraints.positive)
    weights_loc = pyro.param('weights_loc', torch.randn(3))
    weights_scale = pyro.param('weights_scale', torch.ones(3), constraint = constraints.positive)

    a = pyro.sample("a", dist.Normal(a_loc, a_scale))
    b_a = pyro.sample("bA", dist.Normal(weights_loc[0], weights_scale[0]))
    b_r = pyro.sample("bR", dist.Normal(weights_loc[1], weights_scale[1]))
    b_ar = pyro.sample("bAR", dist.Normal(weights_loc[2], weights_scale[2]))
    sigma = pyro.sample("sigma", dist.Normal(sigma_loc, torch.tensor(0.05)))
    mean = a + b_a * is_cont_africa + b_r * ruggedness + b_ar * is_cont_africa * ruggedness

In [ ]:
def summary(samples):
    site_stats = {}
    for site_name, values in samples.items():
        marginal_site = pd.DataFrame(values)
        describe = marginal_site.describe(percentiles = [.05, 0.25, 0.5, 0.75, 0.95]).transpose()
        site_stats[site_name] = describe[["mean", "std", "5%", "25%", "50%", "75%", "95%"]]
    return site_stats


DATA_URL = "https://d2hg8soec8ck9v.cloudfront.net/datasets/rugged_data.csv"
rugged_data = pd.read_csv(DATA_URL, encoding="ISO-8859-1")

df = rugged_data[["cont_africa", "rugged", "rgdppc_2000"]]
df = df[np.isfinite(df.rgdppc_2000)]
df["rgdppc_2000"] = np.log(df["rgdppc_2000"])
train = torch.tensor(df.values, dtype=torch.float)

In [ ]:
from pyro.infer import SVI, Trace_ELBO

svi = SVI(model, guide, optim.Adam({"lr": .05}), loss = Trace_ELBO())

is_cont_africa, ruggedness, log_gdp = train[:, 0], train[:, 1], train[:, 2]
pyro.clear_param_store()
num_iters = 5000 if not smoke_test else 2
for i in range(num_iters):
    elbo = svi.step(is_cont_africa, ruggedness, log_gdp)
    if i % 500 == 0:
        logging.info("ELBO loss: {}".format(elbo))

In [ ]:
elbo

In [ ]:
from pyro.infer import Predictive

num_samples = 1000
predictive = Predictive(model, guide = guide, num_samples = num_samples)
svi_sample = {k: v.reshape(num_samples).detach().cpu().numpy() for k, v in predictive(log_gdp, is_cont_africa, ruggedness).items() if k != "obs"}


In [ ]:
train[:, 2]

In [ ]:
summary(svi_sample)

In [ ]:
for site, values in summary(svi_sample).items():
    print("Site: {}".format(site))
    print(values, "\n")

In [ ]:
from pyro.infer import MCMC, NUTS

nuts_kernel = NUTS(model)
mcmc = MCMC(nuts_kernel, num_samples = 1000, warmup_steps = 200)
mcmc.run(is_cont_africa, ruggedness, log_gdp)

hmc_samples = {k: v.detach().cpu().numpy() for k, v in mcmc.get_samples().items()}

In [ ]:
for site, values in summary(hmc_samples).items():
    print("Site: {}".format(site))
    print(values, "\n")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.datasets import make_regression
import pyro.distributions as dist
from pyro.infer import MCMC, NUTS, Predictive
from pyro.infer.mcmc.util import summary
from pyro.distributions import constraints
import pyro
import torch

pyro.set_rng_seed(101)

%matplotlib inline
%config InlineBackend.figure_format='retina'

In [ ]:
X, y = make_regression(n_features=1, bias=150., noise=5., random_state=108)
X_ = torch.tensor(X, dtype=torch.float)
y_ = torch.tensor((y**3)/100000. + 10., dtype=torch.float)
y_.round_().clamp_(min=0)
plt.scatter(X_, y_)
plt.ylabel('y')
plt.xlabel('x')

In [ ]:
def model(features, counts):
    N, P = features.shape
    scale = pyro.sample("scale", dist.LogNormal(0, 1))
    coef = pyro.sample("coef", dist.Normal(0, scale).expand([P]).to_event(1))
    rate = pyro.deterministic("rate", torch.nn.functional.softplus(coef @ features.T))
    concentration = pyro.sample("concentration", dist.LogNormal(0, 1))
    with pyro.plate("bins", N):
        return pyro.sample("counts", dist.GammaPoisson(concentration, rate), obs = counts)
    

In [ ]:
nuts_kernel = NUTS(model)
mcmc = MCMC(nuts_kernel, num_samples=500)

mcmc.run(X_, y_)

In [ ]:
samples = mcmc.get_samples()
for k, v in samples.items():
    print(f"{k}: {tuple(v.shape)}")

In [ ]:
predictive = Predictive(model, samples)(X_, None)
for k, v in predictive.items():
    print(f"{k}: {tuple(v.shape)}")

In [ ]:
def prepare_counts_df(predictive):
    counts = predictive['counts'].numpy()
    counts_mean = counts.mean(axis=0)
    counts_std = counts.std(axis=0)

    counts_df = pd.DataFrame({
    "feat": X_.squeeze(),
    "mean": counts_mean,
    "high": counts_mean + counts_std,
    "low": counts_mean - counts_std,
    })

    return counts_df.sort_values(by=['feat'])

In [ ]:
counts_df = prepare_counts_df(predictive)
plt.scatter(X_, y_, c='r')
plt.ylabel('y')
plt.xlabel('x')
plt.plot(counts_df['feat'], counts_df['mean'])
plt.fill_between(counts_df['feat'], counts_df['high'], counts_df['low'], alpha=0.5)

In [ ]:
from pyro.infer import SVI, Trace_ELBO
from pyro.optim import Adam
from pyro.infer.autoguide import AutoNormal

def guide(features, counts):
    N, P = features.shape

    scale_param = pyro.param("scale_param", torch.tensor(0.1), constraint = constraints.positive)
    loc_param = pyro.param("loc_param", torch.tensor(0.0))
    scale = pyro.sample("scale", dist.Delta(scale_param))
    coef = pyro.sample("coef", dist.Normal(loc_param, scale).expand([P]).to_event(1))

    concentration_param = pyro.param("concentration_param", torch.tensor(0.1), constraint = constraints.positive)
    concentration = pyro.sample("concentration", dist.Delta(concentration_param))

pyro.clear_param_store()

adam_params = {"lr": 0.005, "betas": (0.90, 0.999)}
optimizer = Adam(adam_params)

svi = SVI(model, guide, optimizer, loss = Trace_ELBO())

n_steps = 5001

for step in range(n_steps):
    loss = svi.step(X_, y_)
    if step % 1000 == 0:
        print('Loss: ', loss)

In [ ]:
list(pyro.get_param_store().items())

In [ ]:
predictive_svi = Predictive(model, guide=guide, num_samples=500)(X_, None)
for k, v in predictive_svi.items():
    print(f"{k}: {tuple(v.shape)}")

In [ ]:
counts_df = prepare_counts_df(predictive_svi)